In [1]:
import scipy.io
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed

dir = 'Data_Enhance/'
items = os.listdir(dir)
items.sort()

def ciriGLDM(x):
    # menghitung ciri dari GLDM
    # h(g|tetha) = probabilitas dif
    # Gradien kontras = sigma (g(h(q|theta)))

    z = np.histogram(np.uint8(x), bins=256, range=(0, 256))[0]
    hg = z / np.sum(z) # Normalized histogram

    # Initialize feature vectors
    gradkon = np.zeros(256) # Gradient contrast
    gradmean = np.zeros(256) # Gradient mean
    idm = np.zeros(256) # Inverse different moment

    # Calculate features for each gray level
    for i in range(256):
        gradkon[i] = (((i-1)**2) * hg[i]) # Gradient contrast for each gray level
        gradmean[i] = (i-1) * hg[i] # Gradient mean for each gray level
        idm[i] = hg[i] / ((i-1)**2 + 1) # Inverse different moment for each gray level

    gradkont = np.sum(gradkon) # Sum of gradient contrasts (ASM)
    gradsm = np.sum(hg**2) # Sum of squared probabilities (second moment)
    gradent = np.sum(hg * np.log10(hg + np.finfo(float).eps)) # Entropy
    gradmeant = np.sum(gradmean) # Sum of gradient means
    idmt = np.sum(idm) # Sum of inverse different moments

    return gradkont, gradsm, gradent, gradmeant, idmt

def process_image(k, new_path, d):
        image = cv2.imread(new_path + k)

        if image is None:
            print("Failed to read image:", k)
            return np.zeros(20)  # Return zeros for failed image

        s = np.shape(image)
        inImg = image.astype(float)

        # matrices
        pro1 = np.zeros(s)  # diff arah 0 derajat
        pro2 = np.zeros(s)  # diff arah 45 derajat
        pro3 = np.zeros(s)  # diff arah 90 derajat
        pro4 = np.zeros(s)  # diff arah 135 derajat

        for i in range(s[0] - d):
            for j in range(s[1] - d):

                if (j + d) <= s[1]:
                    pro1[i, j] = abs(inImg[i, j] - inImg[i, j + d])
                if (i - d) > 0 and (j + d) <= s[1]:
                    pro2[i, j] = abs(inImg[i, j] - inImg[i - d, j + d])
                if (i + d) <= s[0]:
                    pro3[i, j] = abs(inImg[i, j] - inImg[i + d, j])
                if (i - d) > 0 and (j - d) > 0:
                    pro4[i, j] = abs(inImg[i, j] - inImg[i - d, j - d])

        gradkont0, gradsm0, gradent0, gradmeant0, idmt0 = ciriGLDM(pro1)
        ciri0 = [gradkont0, gradsm0, gradent0, gradmeant0, idmt0]

        gradkont45, gradsm45, gradent45, gradmeant45, idmt45 = ciriGLDM(pro2)
        ciri45 = [gradkont45, gradsm45, gradent45, gradmeant45, idmt45]

        gradkont90, gradsm90, gradent90, gradmeant90, idmt90 = ciriGLDM(pro3)
        ciri90 = [gradkont90, gradsm90, gradent90, gradmeant90, idmt90]

        gradkont135, gradsm135, gradent135, gradmeant135, idmt135 = ciriGLDM(pro4)
        ciri135 = [gradkont135, gradsm135, gradent135, gradmeant135, idmt135]

        kontrata = (gradkont0 + gradkont45 + gradkont90 + gradkont135) / 4
        asmrata = (gradsm0 + gradsm45 + gradsm90 + gradsm135) / 4
        entrata = (gradent0 + gradent45 + gradent90 + gradent135) / 4
        meanrata = (gradmeant0 + gradmeant45 + gradmeant90 + gradmeant135) / 4
        idmrata = (idmt0 + idmt45 + idmt90 + idmt135) / 4
        cirirata = [kontrata, asmrata, entrata, meanrata, idmrata]

        ciritotal = np.concatenate((ciri0, ciri45, ciri90, ciri135, cirirata))

        return ciritotal
dir = 'Data_Enhance/'
items = os.listdir(dir)
items.sort()
distance = 5

for dis in tqdm(range(distance), desc='Processing', unit='distance'):
    d = dis + 1
    print(d)
    new_path = "Data_Enhance/"
    
    results = Parallel(n_jobs=-1)(delayed(process_image)(k, new_path, d) for k in tqdm(items, desc='Processing images', unit='image'))

    FiturEEG = np.array(results)
    columns_feature = ['Contrast_{distance}_0', 'Contrast_{distance}_45', 'Contrast_{distance}_90', 'Contrast_{distance}_135',
                       'ASM_{distance}_0', 'ASM_{distance}_45', 'ASM_{distance}_90', 'ASM_{distance}_135',
                       'Entropy_{distance}_0', 'Entropy_{distance}_45', 'Entropy_{distance}_90', 'Entropy_{distance}_135',
                       'Mean_{distance}_0', 'Mean_{distance}_45', 'Mean_{distance}_90', 'Mean_{distance}_135',
                       'IDM_{distance}_0', 'IDM_{distance}_45', 'IDM_{distance}_90', 'IDM_{distance}_135', 'x', 'y','z','o','p']

    columns_feature = [col.replace('{distance}', str(d)) for col in columns_feature]

    features = pd.DataFrame(FiturEEG, columns=columns_feature)
    features.to_csv("GLDM_Histo/{distance}.csv".format(distance=d))


Processing:   0%|                                                                                                                                                                  | 0/5 [00:00<?, ?distance/s]

1



Processing:  20%|██████████████████████████████▊                                                                                                                           | 1/5 [00:23<01:34, 23.72s/distance]

2



Processing:  40%|█████████████████████████████████████████████████████████████▌                                                                                            | 2/5 [00:46<01:09, 23.28s/distance]

...


Processing:  60%|████████████████████████████████████████████████████████████████████████████████████████████▍                                                             | 3/5 [01:08<00:45, 22.66s/distance]

4



Processing:  80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                              | 4/5 [01:29<00:21, 21.88s/distance]

5



Processing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:50<00:00, 22.02s/distance]


In [31]:
import pandas as pd
import os

# List semua file CSV di folder
file_list = [file for file in os.listdir(folder_csv) if file.endswith('.csv')]

# Inisialisasi DataFrame kosong untuk menyimpan data gabungan
combined_data = pd.DataFrame()

# Gabungkan semua kolom dari file-file CSV menjadi satu DataFrame
for file in file_list:
    file_path = os.path.join(folder_csv, file)
    data = pd.read_csv(file_path)
    data.reset_index(drop=True, inplace=True)  # Reset index of each dataframe

    # Hapus kolom 'x', 'y', 'z', 'o', 'p'
    data.drop(['x', 'y', 'z', 'o', 'p'], axis=1, inplace=True)

    combined_data = pd.concat([combined_data, data], axis=1)

# Simpan DataFrame hasil penggabungan ke file CSV di folder yang sama dengan lokasi kode dijalankan
output_file = 'gldm_features_combined.csv'

combined_data.to_csv(output_file, index=False)

# Baca kembali file CSV yang telah disimpan
combined_data_saved = pd.read_csv(output_file)

# Hapus kolom "Unnamed" jika ada
combined_data_saved = combined_data_saved.loc[:, ~combined_data_saved.columns.str.contains('^Unnamed')]# Simpan dataframe tanpa kolom "Unnamed" ke file CSV
combined_data_saved.to_csv(output_file, index=False)
